In [1]:
import os
import tempfile
import numpy as np
import shutil
import urllib
import matplotlib.pyplot as plt
import fsspec

import xarray as xr
from azure.storage.blob import ContainerClient

In [2]:
from dask.distributed import Client
client = Client()
client

/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46185 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:40109 Dashboard: /proxy/8787/status,Cluster Workers: 4 Cores: 4 Memory: 32.00 GiB


### Do setup stuff to get file paths

In [3]:
tempdir = os.path.join(tempfile.gettempdir(), 'goes')
os.makedirs(tempdir, exist_ok=True)

product = 'ABI-L2-MCMIPF'
syear = '2020'; sday='002'; shour = '14'

storage_account_url = 'https://goeseuwest.blob.core.windows.net'
container_name = 'noaa-goes16'
goes_blob_root = storage_account_url + '/' + container_name + '/'

goes_container_client = ContainerClient(account_url=storage_account_url, container_name=container_name, credential=None)

prefix = product + '/' + syear + '/' + sday + '/' 
print('Finding blobs matching prefex: {}'.format(prefix))
generator = goes_container_client.list_blobs(name_starts_with=prefix)
blobs = []
for blob in generator:
    blobs.append(blob.name)


Finding blobs matching prefex: ABI-L2-MCMIPF/2020/002/


In [4]:
fs = fsspec.filesystem("abfs", account_name='goeseuwest')

urllist = ['abfs://noaa-goes16/' + u  for u in blobs]

In [5]:
# fs = fsspec.filesystem("http")

# urllist = [goes_blob_root + u + "#mode=bytes" for u in blobs]

In [6]:
fobjlst = [fs.open(u) for u in urllist]

In [ ]:
%%time
ds = xr.open_mfdataset(fobjlst, combine='nested', concat_dim='t', 
                       coords='minimal', data_vars='minimal', compat='override', 
                       parallel=True)

### Calculate true color
https://unidata.github.io/python-gallery/examples/mapping_GOES16_TrueColor.html

In [27]:
r = ds['CMI_C02'].data; r = np.clip(r, 0, 1)
g = ds['CMI_C03'].data; g = np.clip(g, 0, 1)
b = ds['CMI_C01'].data; b = np.clip(b, 0, 1)

In [28]:
gamma = 2.5; r = np.power(r, 1/gamma); g = np.power(g, 1/gamma); b = np.power(b, 1/gamma)
g_true = 0.45 * r + 0.1 * g + 0.45 * b
g_true = np.clip(g_true, 0, 1)

In [29]:
nt,nx,ny = r.shape
rgb = []
rgb_true = []
for t in range(nt):
    rgb_true.append(np.dstack((r[t,:], g_true[t,:], b[t,:])))
    rgb.append(np.dstack((r[t,:], g[t,:], b[t,:])))

### "True" green

In [30]:
from matplotlib.animation import FuncAnimation

In [46]:
fig = plt.figure(figsize=(7.5,7.5), dpi=100)

import metpy
import cartopy.crs as ccrs


def init():
    dummy_channel = ds.metpy.parse_cf('CMI_C01')
    x = dummy_channel.x; y = dummy_channel.y
    ax = fig.add_subplot(111, projection = dummy_channel.metpy.cartopy_crs)

def update_anim(i): 
    ax.clear()
    ax.imshow(rgb_true[i], origin='upper', extent=(x.min(), x.max(), y.min(), y.max()))
    ax.coastlines()
    ax.add_feature(ccrs.cartopy.feature.BORDERS)
  
# update_anim(1)
animator = FuncAnimation(fig, update_anim, frames=len(urllist), interval=1, init_func=init)
plt.show()

<Figure size 750x750 with 0 Axes>

***
## Processing times:
|Action | Time | Note |
|-------:|:------| :---|
|Open dataset | | |
| Make single plot | | |
| Make animation | | |